In [1]:
import praw
import numpy as np
import nltk
import pandas as pd

## Data Collection

In [2]:
reddit = praw.Reddit(
    client_id = "yL3iXDHMD8L8On0LhYhNIQ",
    client_secret = "tgbVu3DgCbNuPI0z5avZxF01dbTTfQ",
    user_agent = "IndependenceAlive625",
)

In [3]:
subreddit = reddit.subreddit("explainlikeimfive")

In [11]:
submissions_positive = [submission for submission in subreddit.search('flair:"technology" what is the history of')]
submissions_negative = [subreddit.random() for i in range(len(submissions_positive))]

In [13]:
def get_top_comment(s):
    
    scores = [c.score for c in s.comments if not isinstance(c, praw.models.MoreComments)]
    
    if len(scores) == 0:
        return None
    
    return s.comments[np.argmax(scores)].body

In [14]:
data = []
for submission in submissions_positive:
    top_comment = get_top_comment(submission)
    
    if top_comment == None:
        continue
        
    for sent in nltk.sent_tokenize(top_comment):
        data.append((sent, True))
    
for submission in submissions_negative:
    
    top_comment = get_top_comment(submission)
    
    if top_comment == None:
        continue
    
    for sent in nltk.sent_tokenize(top_comment):
        data.append((sent, False))

In [21]:
df = pd.DataFrame(data, columns = ["sentence", "is_instance"])
df.sample(n = 5)

,sentence,is_instance
724,Some are talking about different frequencies (...,True
94,It was chosen because it’s easy to multiply an...,True
479,"This is the tricky part I was talking about, b...",True
885,"[Obviously, this is a gain a simplification as...",False
509,"In the case of our example blockchain, for eac...",True


In [22]:
df.to_csv("data_flair.csv")